In [1]:
pwd

'/home/wsuser/work'

In [2]:
#pip install imutils

In [3]:
#pip install -U scikit-learn

In [4]:
#import sys
#!{sys.executable} -m pip install numpy

In [5]:
#pip install opencv-python

In [6]:
#pip install pickle5

In [7]:
#pip install opencv-python-headless

In [8]:
#pip uninstall -y opencv-contrib-python

In [9]:
#pip uninstall -y opencv-python

In [10]:
#pip install opencv-python

In [11]:
#pip list | grep opencv

# Image Pre-Processing

## Import the necessary Libraries

##### Note: Download scikit-image for skimage 

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os
import pickle5 as pickle

### Path for train and test data

In [13]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='PRVn53XBB5OH8gt-yiTPKTUxVDVGOaxJ4MU9OZAvyHgV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'parkinsondetection-donotdelete-pr-4otyanpngyalea'
object_key = 'spiral.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [14]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

## Quantifying Images

In [15]:
pwd

'/home/wsuser/work'

In [16]:
filenames=os.listdir('/home/wsuser/work/spiral/testing')

In [17]:
def quantify_image(image):
    features = feature.hog(image, orientations=9,
                           pixels_per_cell=(10, 10), 
                           cells_per_block=(2, 2),
                           transform_sqrt=True, 
                           block_norm="L1")
    return features

In [18]:
trainingpath=r"/home/wsuser/work/spiral/training"
testingpath=r"/home/wsuser/work/spiral/testing"

## Loading Train Data and Test Data

In [19]:
def load_split(path):
    imagePaths = list(paths.list_images(path))
    data = []
    labels = []
    
    for imagePath in imagePaths:
        label = imagePath.split(os.path.sep)[-2]
        
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200, 200))

        image=cv2.threshold(image,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        
        features = quantify_image(image)

        data.append(features)
        labels.append(label)

    return (np.array(data), np.array(labels))


### Load the train and test data

In [20]:
print("[INFO] loading data...")
(X_train, y_train) = load_split(trainingpath)
(X_test, y_test) = load_split(testingpath)

[INFO] loading data...


## Label Encoding

In [21]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
print(X_train.shape,y_train.shape)

(72, 12996) (72,)


# Model Building

## Training The Model

In [22]:
print("[INFO] training model")
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

[INFO] training model


RandomForestClassifier()

## Testing The Model

In [23]:
testingpath=list(paths.list_images(testingpath))
idxs=np.arange(0,len(testingpath))
idxs=np.random.choice(idxs,size=(25,),replace=False)
images=[]

In [24]:
label_1=[]
for i in idxs:
    image=cv2.imread(testingpath[i])
    output=image.copy()
        
    # load the input image,convert to grayscale and resize
    
    output=cv2.resize(output,(128,128))
    image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image=cv2.resize(image,(200,200))
    image=cv2.threshold(image,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    #quantify the image and make predictions based on the  extracted feature using last trained random forest
    features=quantify_image(image)
    preds=model.predict([features])
    label=le.inverse_transform(preds)[0]
    #the set of output images
    if label=="healthy":
        color=(0,255,0)
    else:
        color=(0,0,255)
  
    label_1.append(label)
    cv2.putText(output,label,(3,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,2)
    images.append(output)

#creating a montage
montage=build_montages(images,(128,128),(5,5))[0]

In [25]:
print(label_1)

['healthy', 'parkinson', 'healthy', 'healthy', 'parkinson', 'healthy', 'healthy', 'healthy', 'healthy', 'healthy', 'parkinson', 'healthy', 'healthy', 'parkinson', 'healthy', 'parkinson', 'parkinson', 'healthy', 'healthy', 'parkinson', 'parkinson', 'parkinson', 'parkinson', 'healthy', 'healthy']


## Model Evaluation

In [26]:
predictions = model.predict(X_test)

cm = confusion_matrix(y_test, predictions).flatten()
print(cm)
(tn, fp, fn, tp) = cm
accuracy = (tp + tn) / float(cm.sum())
print(accuracy)

[14  1  4 11]
0.8333333333333334


## Save The Model

In [27]:
pickle.dump(model,open('parkinson.pkl','wb')) 

In [28]:
ls -1

parkinson.pkl
spiral/


In [31]:
!tar -zcvf parkinson-model_new.tgz parkinson.pkl

parkinson.pkl


In [32]:
ls -1

parkinson-model_new.tgz
parkinson.pkl
spiral/


In [34]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 20.8 MB/s eta 0:00:01


In [40]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [41]:
pip install ibm-watson-machine-learning

Note: you may need to restart the kernel to use updated packages.


In [45]:
#Replace the credentials that you got from Watson Machine Learning service 
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                    "apikey":"LTDYdKh2PQUmhEDewTt06vMt9fFKObC4beimefuTak93"
                  }
client=APIClient (wml_credentials)

In [46]:
client=APIClient (wml_credentials)

In [49]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [50]:
space_uid= guid_from_space_name(client, 'Parkinsons_deployment')
print("Space UID = " + space_uid)

Space UID = 42160dda-5232-4791-874f-c3341bf34b05


In [51]:
client.set.default_space(space_uid)

'SUCCESS'

In [52]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [89]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9") 
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [71]:
!pip install keras==2.2.4

     |████████████████████████████████| 312 kB 22.6 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 16.2 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.
autoai-ts-libs 1.1.9 requires scikit-learn<1.1,>=1.0.2; python_version >= "3.9", but you have scikit-learn 1.1.3 which is incompatible.


In [91]:
model_details=client.repository.store_model(model='parkinson-model_new.tgz', meta_props={
client.repository.ModelMetaNames.NAME:"Parkinsons", 
client.repository.ModelMetaNames.TYPE: "scikit-learn_1.0",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
                                           )

model_id=client.repository.get_model_id(model_details)

In [92]:
model_id

'59a2fbdb-ee8a-46f3-bcc7-817d6024af15'

In [97]:
client.repository.download(model_id,'parkinsons_model.tar.gz')

Successfully saved model content to file: 'parkinsons_model.tar.gz'


'/home/wsuser/work/parkinsons_model.tar.gz'